In [1]:
import pandas as pd
import re
import numpy as np
from collections import defaultdict
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

In [4]:
dataset = pd.read_csv('PS1.1A_training_data.txt', delimiter='\t', header=None, error_bad_lines=False)
Test_Data = pd.read_csv('PS1.1A_test_data.txt', delimiter='\t', header=None, error_bad_lines=False)

b'Skipping line 2560: expected 5 fields, saw 6\n'


In [5]:
#renaming all the columns
dataset = dataset.rename(columns = {0: 'id'})
dataset = dataset.rename(columns = {1: 'text'})
dataset = dataset.rename(columns = {2: 'sentiment'})

Test_Data = Test_Data.rename(columns = {0: 'id'})
Test_Data = Test_Data.rename(columns = {1: 'text'})
Test_Data = Test_Data.rename(columns = {2: 'sentiment'})

In [6]:
dataset.head(5)

,id,text,sentiment,3,4
0,0,This is definitely a must have if your state d...,POSITIVE,NONE,GENRE_B
1,1,It's a great place and I highly recommend it.,POSITIVE,NONE,GENRE_B
2,2,"I will see the doctors, take some blood tests ...",NEUTRAL,GOING_TO_PLACES,GENRE_A
3,3,I can tell you about having my phone and elect...,NEGATIVE,MONEY_ISSUE,GENRE_A
4,4,Their steaks are 100% recommended!,POSITIVE,NONE,GENRE_B


In [9]:
#cleaning datasets
def Cleaning_text(dataset):
    stopwords_english = stopwords.words('english')
    
    # pruning neutral sentiment
    dataset = dataset[dataset.sentiment != 'NEUTRAL']
    
    for column in dataset.columns:
        if column != 'id':
             # removing punctuation, lowercase and remove stopwords
            dataset[column] = dataset[column].str.replace(r'[^\w\s]', '')
            dataset[column] = dataset[column].str.lower()
            dataset[column] = dataset[column].apply(lambda x: " ".join([i for i in x.split() if i not in stopwords_english]))

    return dataset 

In [10]:
dataset = Cleaning_text(dataset)
Test_Data = Cleaning_text(Test_Data)

<ipython-input-9-9346da5be0c1>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset[column] = dataset[column].str.replace(r'[^\w\s]', '')
<ipython-input-9-9346da5be0c1>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[column] = dataset[column].str.replace(r'[^\w\s]', '')
<ipython-input-9-9346da5be0c1>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[column] = dataset[column].str.lower()
<ipython-input-9-9346da5be0c1>:12: SettingWithCop

In [11]:
dataset.head(5)

,id,text,sentiment,3,4
0,0,definitely must state allow cell phone usage d...,positive,none,genre_b
1,1,great place highly recommend,positive,none,genre_b
3,3,tell phone electricity gas cut couldnt pay bills,negative,money_issue,genre_a
4,4,steaks 100 recommended,positive,none,genre_b
5,5,billed thousands dollars said still feeling sh...,negative,money_issue,genre_a


In [12]:
text_train, text_test, label_train, label_test = train_test_split(dataset['text'], dataset['sentiment'], test_size=0.3, random_state=5)

In [13]:
print(text_train)

1363                enough said remarkable animation film
2436                       lot websites rating good phone
1517               punishment park brilliant piece cinema
1259                                            movie bad
2418    elias koteasjack palance play good roles angel...
                              ...                        
1121    always wanted remain resident france pay taxes...
2308    probably would better director hadnt spent mov...
1543          didnt see far along late change bad haircut
1869    posted detailed comments grey black phone fire...
2451    absolutely problem headset linking 8530 blackb...
Name: text, Length: 1648, dtype: object


In [33]:
class Naive_Bayes:
    def __init__(self):
        self.log_prior = defaultdict(int)
        self.V = []
        self.bigdoc = defaultdict(list)
        self.unique_classes = set()
        self.loglikelihood = defaultdict(defaultdict)
        
    def Count(self):
        counts = {}
        for each_class in list(self.bigdoc.keys()):
            text = self.bigdoc[each_class]
            counts[each_class] = defaultdict(int)
            for words in text:
                word = words.split(" ")
                for each_word in word:
                    counts[each_class][each_word] += 1
        return counts
        
    def Train_Naive_Bayes(self, text_train, label_train):
        Ndoc = len(text_train)
        self.unique_classes = set(label_train)
            
        vocabulary = set()
        for each in text_train:
            for each_word in each.split(" "):
                vocabulary.add(each_word)
        self.V = vocabulary
            
        # BIGDOC:
        for a, b in zip(text_train, label_train):
            self.bigdoc[b].append(a)
                
        self.word_count = self.Count()
        for classes in self.unique_classes:
            Nc = float(sum(label_train == classes))
            self.log_prior[classes] = np.log(Nc / Ndoc)
                
            total_count = 0
            for word in self.V:
                total_count += self.word_count[classes][word]
                    
            for word in self.V:
                count = self.word_count[classes][word]
                self.loglikelihood[classes][word] = np.log((count + 1) / (total_count + 1))
        return self.log_prior, self.loglikelihood
    
    def Test_Naive_Bayes(self, Test_Data, log_prior, loglikelihood):
        sums = defaultdict(int)
        labels = []
        for row in Test_Data:
            for classes in self.unique_classes:
                sums[classes] = log_prior[classes]
                words = nltk.word_tokenize(row)
                for word in words:
                    if word in self.V:
                        sums[classes] += loglikelihood[classes][word]

            label_and_likelihoods = [(classes, sums[classes]) for classes in self.unique_classes]
            best_label = list(map(max, zip(*label_and_likelihoods)))[0]
            labels.append(best_label)

        return labels
        

In [34]:
train_log_prior = defaultdict(int)
train_loglikelihood = defaultdict(defaultdict)
test_pred = []
Naive_Bayes = Naive_Bayes()
train_log_prior, train_loglikelihood = Naive_Bayes.Train_Naive_Bayes(text_train, label_train)
train_pred = Naive_Bayes.Test_Naive_Bayes(text_train, train_log_prior, train_loglikelihood)
print(train_pred)

['positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'po

In [35]:
test_pred = Naive_Bayes.Test_Naive_Bayes(Test_Data['text'], train_log_prior, train_loglikelihood)
y_true=Test_Data['sentiment']
print(test_pred)

['positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'po

In [27]:
# Accuracy
from sklearn.metrics import accuracy_score
print("Accuracy:",accuracy_score(y_true, test_pred))
# Recall
from sklearn.metrics import recall_score
print('recall: ',recall_score(y_true, test_pred, average=None))
# Precision
from sklearn.metrics import precision_score
print('precision score: ', precision_score(y_true, test_pred, average=None))

Accuracy: 0.49142550911039656
recall:  [0. 1.]
precision score:  [0.         0.49142551]


/.autofs/tools/spack/var/spack/environments/reu-sep-21062801/.spack-env/view/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true, test_pred)

array([[  0, 949],
       [  0, 917]])

In [29]:
#########################################################################################################
#using scikit-learn#(Bonus-part)
#########################################################################################################

In [36]:
independent = ["text"]
X = dataset[independent]

In [37]:
X

,text
0,definitely must state allow cell phone usage d...
1,great place highly recommend
3,tell phone electricity gas cut couldnt pay bills
4,steaks 100 recommended
5,billed thousands dollars said still feeling sh...
...,...
2554,recognized happening feeling going tsunami
2555,one better buffets
2556,theres elegant way say fired
2557,thats go pay bucks movies taken away escape said


In [38]:
y=dataset['sentiment']
y

0       positive
1       positive
3       negative
4       positive
5       negative
          ...   
2554    negative
2555    positive
2556    negative
2557    positive
2558    positive
Name: sentiment, Length: 2355, dtype: object

In [39]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
text_counts = cv.fit_transform(X['text'])
text_counts

<2355x6295 sparse matrix of type '<class 'numpy.int64'>'
	with 16632 stored elements in Compressed Sparse Row format>

In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(text_counts.toarray(), y, test_size=0.25, random_state=5)

In [41]:
from sklearn.naive_bayes import MultinomialNB

In [42]:
MNB = MultinomialNB()
MNB.fit(X_train, Y_train)


MultinomialNB()

In [43]:
from sklearn import metrics
predicted = MNB.predict(X_test)
accuracy_score = metrics.accuracy_score(predicted, Y_test)

In [46]:
print(str("Accuracy " '{:04.2f}'.format(accuracy_score*100))+'%')

Accuracy 77.93%
